In [112]:
import requests_html as rh
import json
import pandas as pd

In [124]:
sess = rh.HTMLSession()
url = 'https://www.nfl.com/teams/'
r = sess.get(url)

In [ ]:
d3-o-media-object__link d3-o-button nfl-o-cta nfl-o-cta--primary
d3-o-media-object__cta nfl-c-custom-promo__cta
d3-o-media-object__cta nfl-c-custom-promo__cta

In [137]:
table = r.html.find('main#main-content', first = True)
table_urls = table.find('div.d3-o-media-object__cta')
team_route_list = [x for x in list(table_urls[0].links) if x.startswith('/teams/')]

In [330]:
team_route_list

['/teams/new-orleans-saints/',
 '/teams/cleveland-browns/',
 '/teams/jacksonville-jaguars/',
 '/teams/tampa-bay-buccaneers/',
 '/teams/green-bay-packers/',
 '/teams/philadelphia-eagles/',
 '/teams/los-angeles-rams/',
 '/teams/chicago-bears/',
 '/teams/new-york-jets/',
 '/teams/buffalo-bills/',
 '/teams/new-york-giants/',
 '/teams/cincinnati-bengals/',
 '/teams/pittsburgh-steelers/',
 '/teams/atlanta-falcons/',
 '/teams/arizona-cardinals/',
 '/teams/las-vegas-raiders/',
 '/teams/los-angeles-chargers/',
 '/teams/miami-dolphins/',
 '/teams/denver-broncos/',
 '/teams/san-francisco-49ers/',
 '/teams/new-england-patriots/',
 '/teams/tennessee-titans/',
 '/teams/houston-texans',
 '/teams/washington-commanders/',
 '/teams/dallas-cowboys/',
 '/teams/kansas-city-chiefs/',
 '/teams/minnesota-vikings/',
 '/teams/detroit-lions/',
 '/teams/carolina-panthers/',
 '/teams/baltimore-ravens/',
 '/teams/seattle-seahawks/',
 '/teams/indianapolis-colts']

In [239]:
baseUrl = 'https://www.nfl.com'
test_team = team_route_list[0]
test_team += 'roster'


test_team_url = baseUrl + test_team
test_r = sess.get(test_team_url)
test_table = test_r.html.find('main#main-content', first = True)
test_team_info = test_table.find('div.nfl-c-team-header__info', first = True)
test_team_name = test_team_info.find('div.nfl-c-team-header__title', first = True).element.text
test_team_name

team_roster_div = test_table.find('div.d3-l-col__col-12', first = True)
team_roster_div = team_roster_div.find('div.nfl-o-roster', first = True)
team_roster_table = team_roster_div.find('table', first = True)
team_roster_table = team_roster_table.find('tbody', first = True)
team_roster_table = team_roster_table.find('tr')

print(team_roster_table[0].find('td')[0].text.split('\n')[0:8])


['Paulson Adebo', '29', 'CB', 'ACT', '73', '192', '2', 'Stanford']


In [243]:
print(team_roster_table[0].find('td')[0].text.split('\n')[1].isdigit())

True


In [ ]:
# team_roster_table[0].find('td')[0].text.split('\n')[0:8]
# team_roster_table = [x for x in team_roster_table if x.find('td', first = True).element.text != '']
# print(team_roster_table[0].find('td', first = True).element.text)

In [317]:
t

['Paulson Adebo', 'CB', 'ACT', '73', '192', '2', 'Stanford', 'random shit']

In [338]:
baseUrl = 'https://www.nfl.com'

# output = {
#     'players': []
# }


def get_team_routes() -> list:
    """
    Get the team routes from the NFL.com website.
    """
    sess = rh.HTMLSession()
    url = 'https://www.nfl.com/teams/'
    r = sess.get(url)

    table = r.html.find('main#main-content', first = True)
    table_urls = table.find('div.d3-o-media-object__cta')
    team_route_list = [x for x in list(table_urls[0].links) if x.startswith('/teams/')]
    
    return team_route_list

def player_row_cleanse(row: list) -> list:
    """
    Small clean up the player row data from the NFL.com website.
    """

    if not row[1].isdigit():
        row.insert(1, None)
    if not row[2].isalpha():
        row.insert(2, None)
    if not row[3].isalpha():
        row.insert(3, None)
    if not row[4].isdigit():
        row.insert(4, None)
    if not row[5].isdigit():
        row.insert(5, None)
    if not row[6].isdigit():
        row.insert(6, None)
        
    # remove last element (make sure row is 8 elements long)
    while len(row) > 8:
        del row[-1]
    return row


def get_team_roster(team_route: str) -> pd.DataFrame:
    """
    Get the team roster from the NFL.com website.
    """

    team_url = baseUrl + team_route + 'roster' if team_route.endswith('/') else baseUrl + team_route + '/roster'
    team_r = sess.get(team_url)
    
    team_table = team_r.html.find('main#main-content', first = True)
    team_info = team_table.find('div.nfl-c-team-header__info', first = True)
    team_name = team_info.find('div.nfl-c-team-header__title', first = True).element.text
    
    team_roster_div = team_table.find('div.d3-l-col__col-12', first = True)
    team_roster_div = team_roster_div.find('div.nfl-o-roster', first = True)
    
    team_roster_table = team_roster_div.find('table', first = True)
    team_roster_table = team_roster_table.find('tbody', first = True)
    team_roster_table = team_roster_table.find('tr')
    
    # list of lists
    team_roster = [player_row_cleanse(x.find('td')[0].text.split('\n')[0:8]) for x in team_roster_table]
    
    # storing as pd dataframe for now (might need to push list of lists into db)
    team_roster = pd.DataFrame(team_roster, columns = ['name', 'number', 'position', 'status', 'height', 'weight', 'experience', 'college'])
    team_roster['team'] = team_name
    return team_roster

def player_row_to_dict(row: list) -> dict:
    """
    Convert the player row data to a dict.
    """
    d = {
        'name': row[1],
        'jersey_number': row[2],
        'position': row[3],
        'status': row[4],
        'height': row[5],
        'weight': row[6],
        'experience': row[7],
        'college': row[8],
        'team': row[0]
    }
    
    return d

def get_all_team_rosters() -> pd.DataFrame:
    """
    Get all the team rosters from the NFL.com website.
    """
    
    team_routes = get_team_routes()

    # may need to change this to a list of lists (for db)
    team_rosters = [get_team_roster(x) for x in team_routes]
    team_rosters = pd.concat(team_rosters)
    return team_rosters



def get_all_team_rosters_to_dict() -> list:
    """
    Get all the team rosters from the NFL.com website.
    """
    
    team_routes = get_team_routes()

    # may need to change this to a list of lists (for db)
    team_rosters = [get_team_roster(x) for x in team_routes]
    team_rosters = pd.concat(team_rosters)
    team_rosters = team_rosters.to_dict('records')
    return team_rosters

In [339]:
df = get_team_roster(team_route_list[0])

[['Paulson Adebo', '29', 'CB', 'ACT', '73', '192', '2', 'Stanford'], ['Josh Andrews', '68', 'C', 'ACT', '74', '298', '7', 'Oregon State'], ['Kawaan Baker', '87', 'WR', 'TRD', '73', '215', '1', 'South Alabama'], ['Zack Baun', '53', 'LB', 'ACT', '75', '225', '3', 'Wisconsin'], ['Jordan Brown', '38', 'DB', 'CUT', '72', '197', '1', 'South Dakota State'], ['Marquez Callaway', '1', 'WR', 'ACT', '74', '204', '3', 'Tennessee'], ['Taco Charlton', '54', 'DE', 'TRD', '78', '270', '6', 'Michigan'], ['Andy Dalton', '14', 'QB', 'ACT', '74', '220', '12', 'TCU'], ['Marcus Davenport', '92', 'DE', 'ACT', '78', '265', '5', 'Texas-San Antonio'], ['Demario Davis', '56', 'OLB', 'ACT', '74', '248', '11', 'Arkansas State'], ['Wyatt Davis', '70', 'G', 'TRD', '76', '315', '2', 'Ohio State'], ['Drew Desjarlais', '61', 'G', 'DEV', '74', '313', '1', 'University of Windsor'], ["Dai'Jean Dixon", '84', 'WR', 'CUT', '76', '200', None, 'R'], ['Andrew Dowell', '50', 'LB', 'ACT', '73', '225', '2', 'Michigan State'], ['Ya

In [340]:
df

,name,number,position,status,height,weight,experience,college,team
0,Paulson Adebo,29,CB,ACT,73,192,2,Stanford,New Orleans Saints
1,Josh Andrews,68,C,ACT,74,298,7,Oregon State,New Orleans Saints
2,Kawaan Baker,87,WR,TRD,73,215,1,South Alabama,New Orleans Saints
3,Zack Baun,53,LB,ACT,75,225,3,Wisconsin,New Orleans Saints
4,Jordan Brown,38,DB,CUT,72,197,1,South Dakota State,New Orleans Saints
...,...,...,...,...,...,...,...,...,...
92,Zach Wood,49,LS,ACT,75,255,6,SMU,New Orleans Saints
93,Isaac Yiadom,None,CB,DEV,73,188,5,Boston College,New Orleans Saints
94,Kenny Young,None,ILB,CUT,73,234,5,UCLA,New Orleans Saints
95,Landon Young,67,OT,ACT,79,321,2,Kentucky,New Orleans Saints


In [331]:
baseUrl = 'https://www.nfl.com'

output = {
    'players': []
}


for team in team_route_list:
    team_url = baseUrl + team + 'roster' if team.endswith('/') else baseUrl + team + '/roster'
    team_r = sess.get(team_url)
    team_table = team_r.html.find('main#main-content', first = True)
    team_info = team_table.find('div.nfl-c-team-header__info', first = True)
    
#     print('team url', team_url)
#     team_name = team_info.find('div.nfl-c-team-header__title', first = True).element.text
#     team_roster_div = team_table.find('div.d3-l-col__col-12', first = True)
#     team_roster_div = team_roster_div.find('div.nfl-o-roster', first = True)
#     team_roster_table = team_roster_div.find('table', first = True)
#     team_roster_table = team_roster_table.find('tbody', first = True)
#     team_roster_table = team_roster_table.find('tr')

    
#     for row in team_roster_table:
#         row = row.find('td', first = True)
#         row_text = player_row_cleanse(row.text.split('\n')[0:8])
#         row = [team_name] + row_text
#         output['players'].append(
#             {
#                 'name': row[1],
#                 'jersey_number': row[2],
#                 'position': row[3],
#                 'status': row[4],
#                 'height': row[5],
#                 'weight': row[6],
#                 'experience': row[7],
#                 'college': row[8],
#                 'team': row[0]
#             }
#         )

team url https://www.nfl.com/teams/new-orleans-saints/roster
team url https://www.nfl.com/teams/cleveland-browns/roster
team url https://www.nfl.com/teams/jacksonville-jaguars/roster
team url https://www.nfl.com/teams/tampa-bay-buccaneers/roster
team url https://www.nfl.com/teams/green-bay-packers/roster
team url https://www.nfl.com/teams/philadelphia-eagles/roster
team url https://www.nfl.com/teams/los-angeles-rams/roster
team url https://www.nfl.com/teams/chicago-bears/roster
team url https://www.nfl.com/teams/new-york-jets/roster
team url https://www.nfl.com/teams/buffalo-bills/roster
team url https://www.nfl.com/teams/new-york-giants/roster
team url https://www.nfl.com/teams/cincinnati-bengals/roster
team url https://www.nfl.com/teams/pittsburgh-steelers/roster
team url https://www.nfl.com/teams/atlanta-falcons/roster
team url https://www.nfl.com/teams/arizona-cardinals/roster
team url https://www.nfl.com/teams/las-vegas-raiders/roster
team url https://www.nfl.com/teams/los-angeles

In [332]:
output

{'players': [{'name': 'Paulson Adebo',
   'jersey_number': '29',
   'position': 'CB',
   'status': 'ACT',
   'height': '73',
   'weight': '192',
   'experience': '2',
   'college': 'Stanford',
   'team': 'New Orleans Saints'},
  {'name': 'Josh Andrews',
   'jersey_number': '68',
   'position': 'C',
   'status': 'ACT',
   'height': '74',
   'weight': '298',
   'experience': '7',
   'college': 'Oregon State',
   'team': 'New Orleans Saints'},
  {'name': 'Kawaan Baker',
   'jersey_number': '87',
   'position': 'WR',
   'status': 'TRD',
   'height': '73',
   'weight': '215',
   'experience': '1',
   'college': 'South Alabama',
   'team': 'New Orleans Saints'},
  {'name': 'Zack Baun',
   'jersey_number': '53',
   'position': 'LB',
   'status': 'ACT',
   'height': '75',
   'weight': '225',
   'experience': '3',
   'college': 'Wisconsin',
   'team': 'New Orleans Saints'},
  {'name': 'Jordan Brown',
   'jersey_number': '38',
   'position': 'DB',
   'status': 'CUT',
   'height': '72',
   'weigh

In [ ]:
nfl-o-roster__player-name nfl-o-cta--link
nfl-o-roster__player-name nfl-o-cta--link



In [180]:
team_route_list[0] + 'roster'

'/teams/new-orleans-saints/roster'

In [178]:
len(team_route_list)

32

In [106]:
# looking to get players from team rosters
# by first getting each team link then using it to get every player on roster

for i in range (0, len(table_urls)):
    print(table_urls[i].text.split('\n')[0].strip())

Patrick MahomesKC
Tom BradyTB
Josh AllenBUF
Aaron RodgersGB
Joe BurrowCIN
Geno SmithSEA
Justin HerbertLAC
Kirk CousinsMIN
Trevor LawrenceJAX
Jared GoffDET
Tua TagovailoaMIA
Matt RyanIND
Jalen HurtsPHI
Kyler MurrayARI
Derek CarrLV
Jacoby BrissettCLE
Russell WilsonDEN
Davis MillsHOU
Jimmy GaroppoloSF
Matthew StaffordLAR
Lamar JacksonBAL
Marcus MariotaATL
Ryan TannehillTEN
Daniel JonesNYG
Andy DaltonNO
Carson WentzWSH
Justin FieldsCHI
Zach WilsonNYJ
Kenny PickettPIT
Mac JonesNE
Baker MayfieldCAR
Cooper RushDAL
Joe FlaccoNYJ
Jameis WinstonNO
Dak PrescottDAL
Taylor HeinickeWSH
Mitch TrubiskyPIT
Bailey ZappeNE
PJ WalkerCAR
Teddy BridgewaterMIA
Sam EhlingerIND
Skylar ThompsonMIA
Colt McCoyARI
Brett RypienDEN
John WolfordLAR
Trey LanceSF
Malik WillisTEN
Taysom HillNO
Jordan LoveGB
Jarrett StidhamLV
CMP
ATT
CMP%
YDS
AVG
YDS/G
LNG
TD
INT
SACK
SYL
QBR
RTG
Show More
Terms of Use
Privacy Policy
Your California Privacy Rights
Children's Online Privacy Policy
Interest-Based Ads
About Nielsen Measurem

In [88]:
names_list = []

for i in 


name = table_urls[0].text.split('\n')[0].strip()

temp = name.split(' ')
count = 0
team_stack = []


for i in range(len(temp[1])-1, 0, -1):
    if temp[1][i].isupper():
        team_stack.append(temp[1][i])
        count += 1
    else:
        break

short_team = ''.join(team_stack[::-1])
name = name[:-count]







# team_stack


# for i in temp[-1]:
#     print(i)


# if temp[-1].isupper():
#     name = ' '.join(temp[:-1])
#     team = temp[-1]
# else:
#     name = name
#     team = None

# print(temp[-1])
# name = name.split(' ')

# name = ' '.join(name)


KC


In [ ]:
output = {"players"}

In [52]:
name

'Patrick MahomesK'

In [12]:
# table_urls = [x.absolute_links.pop() for x in table_urls if 'player/_/id/' in x]

TypeError: argument of type 'Element' is not iterable

In [8]:
table_urls

['https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/espnplus/?om-navmethod=topnav',
 'https://www.espn.com/nfl/stats/player/_/stat/receiving',
 'https://www.espn.com/nfl/stats/player/_/stat/receiving',
 'https://www.espn.com/nfl/stats/player/_/stat/receiving',
 'https://www.espn.com/nfl/stats/player/_/stat/receiving',
 'https://www.espn.com/nfl/stats/player/_/stat/receiving',
 'https://www.espn.com/nfl/stats/player/_/stat/receiving',
 'https://www.espn.com/nfl/stats/player/_/stat/receiving',
 'https://www.espn.com/nfl/stats/play